In [2]:

# Imports
import pandas as pd
import numpy as np
import re, hashlib, unicodedata
from pathlib import Path
INPUT_DIR = Path("./data/raw")     # place football-data CSVs here (e.g., I1.csv, E0.csv, etc.)
OUTPUT_MERGED = Path("./data/b365_merged.csv")  # final table

pd.set_option("display.max_columns", 120)


In [3]:

def parse_date_series(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors="coerce", dayfirst=True)

def make_match_id(row) -> str:
    dt = row.get("Date")
    if not isinstance(dt, pd.Timestamp):
        dt = pd.to_datetime(dt, errors="coerce", dayfirst=True)
    date_str = dt.strftime("%Y-%m-%d") if isinstance(dt, pd.Timestamp) else ""
    key = f"{row.get('Div','')}|{date_str}|{row.get('HomeTeam','')}|{row.get('AwayTeam','')}"
    return hashlib.sha1(key.encode("utf-8")).hexdigest()[:16]

def norm_team(s):
    if pd.isna(s): return s
    s = str(s)
    s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    s = s.lower().replace("&","and").replace("."," ").replace("-"," ")
    return " ".join(s.split())

def detect_b365_columns(columns):
    cols = set(columns)
    pre_1x2 = {k: k for k in ["B365H", "B365D", "B365A"] if k in cols}
    close_1x2 = {k: k for k in ["B365CH", "B365CD", "B365CA"] if k in cols}
    pre_ou = {}
    if "B365>2.5" in cols: pre_ou["B365>2.5"] = "B365>2.5"
    if "B365<2.5" in cols: pre_ou["B365<2.5"] = "B365<2.5"
    close_ou = {}
    if "B365C>2.5" in cols: close_ou["B365C>2.5"] = "B365C>2.5"
    if "B365C<2.5" in cols: close_ou["B365C<2.5"] = "B365C<2.5"
    pre_ah = {k: k for k in ["B365AHH", "B365AHA", "B365AH"] if k in cols}
    close_ah = {k: k for k in ["B365CAHH", "B365CAHA", "B365CAH"] if k in cols}
    return {"pre_1x2": pre_1x2, "close_1x2": close_1x2, "pre_ou": pre_ou, "close_ou": close_ou, "pre_ah": pre_ah, "close_ah": close_ah}

def implied_probs_1x2(df):
    cols_close = ["close_home","close_draw","close_away"]
    cols_pre   = ["pre_home","pre_draw","pre_away"]
    if set(cols_close).issubset(df.columns) and df[cols_close].notna().any(axis=None):
        cols = cols_close
    else:
        cols = cols_pre
    inv = 1.0 / df[cols]
    inv_sum = inv.sum(axis=1)
    probs = inv.div(inv_sum, axis=0).rename(columns={cols[0]:"p_home", cols[1]:"p_draw", cols[2]:"p_away"})
    return probs

def implied_prob_over25(df):
    if {"close_over2_5","close_under2_5"}.issubset(df.columns) and df[["close_over2_5","close_under2_5"]].notna().any(axis=None):
        inv_over = 1.0 / df["close_over2_5"]
        inv_under = 1.0 / df["close_under2_5"]
    else:
        inv_over = 1.0 / df.get("pre_over2_5", pd.Series(index=df.index, dtype=float))
        inv_under = 1.0 / df.get("pre_under2_5", pd.Series(index=df.index, dtype=float))
    return (inv_over / (inv_over + inv_under)).rename("p_over25")


In [4]:
files = sorted(INPUT_DIR.glob("*.csv"))
if not files:
    raise FileNotFoundError(f"No CSVs found in {INPUT_DIR}")
len(files), [f.name for f in files][:10]


(57,
 ['D1 (1).csv',
  'D1 (10).csv',
  'D1 (2).csv',
  'D1 (3).csv',
  'D1 (4).csv',
  'D1 (5).csv',
  'D1 (6).csv',
  'D1 (7).csv',
  'D1 (8).csv',
  'D1 (9).csv'])

In [5]:
matches_frames = []
odds_frames = []
corners_frames = []

for f in files:
    try:
        df = pd.read_csv(f, encoding="latin1")
    except UnicodeDecodeError:
        df = pd.read_csv(f, encoding="utf-8", errors="ignore")
    df.columns = [c.strip() for c in df.columns]
    df["__source_file"] = f.name

    if not {"Date","HomeTeam","AwayTeam"}.issubset(df.columns):
        continue

    df["Date"] = parse_date_series(df["Date"])

    for col in ["Div","FTHG","FTAG","FTR","HTHG","HTAG","HTR","Attendance","Referee","Season"]:
        if col not in df.columns:
            df[col] = np.nan

    m = df[["Div","Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HTHG","HTAG","HTR","Attendance","Referee","Season","__source_file"]].copy()
    m["match_id"] = m.apply(make_match_id, axis=1)
    m["jn_date"] = m["Date"].dt.strftime("%Y-%m-%d")
    m["jn_home"] = m["HomeTeam"].apply(norm_team)
    m["jn_away"] = m["AwayTeam"].apply(norm_team)
    m["jn_key"]  = m["jn_date"] + "|" + m["jn_home"] + "|" + m["jn_away"]
    matches_frames.append(m)

    bcols = detect_b365_columns(df.columns)
    if bcols["pre_1x2"] or bcols["close_1x2"]:
        o = pd.DataFrame({
            "match_id": m["match_id"],
            "jn_key": m["jn_key"],
            "pre_home": pd.to_numeric(df.get("B365H", pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "pre_draw": pd.to_numeric(df.get("B365D", pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "pre_away": pd.to_numeric(df.get("B365A", pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "close_home": pd.to_numeric(df.get("B365CH", pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "close_draw": pd.to_numeric(df.get("B365CD", pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "close_away": pd.to_numeric(df.get("B365CA", pd.Series(index=df.index, dtype=float)), errors="coerce"),
        })
        odds_frames.append(o)

    if bcols["pre_ou"] or bcols["close_ou"]:
        ou = pd.DataFrame({
            "match_id": m["match_id"],
            "jn_key": m["jn_key"],
            "pre_over2_5": pd.to_numeric(df.get("B365>2.5", pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "pre_under2_5": pd.to_numeric(df.get("B365<2.5", pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "close_over2_5": pd.to_numeric(df.get("B365C>2.5", pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "close_under2_5": pd.to_numeric(df.get("B365C<2.5", pd.Series(index=df.index, dtype=float)), errors="coerce"),
        })
        odds_frames.append(ou)

    if bcols["pre_ah"] or bcols["close_ah"]:
        ah = pd.DataFrame({
            "match_id": m["match_id"],
            "jn_key": m["jn_key"],
            "pre_line_home": pd.to_numeric(df.get("B365AH",  pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "close_line_home": pd.to_numeric(df.get("B365CAH", pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "pre_home_ah": pd.to_numeric(df.get("B365AHH",  pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "pre_away_ah": pd.to_numeric(df.get("B365AHA",  pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "close_home_ah": pd.to_numeric(df.get("B365CAHH", pd.Series(index=df.index, dtype=float)), errors="coerce"),
            "close_away_ah": pd.to_numeric(df.get("B365CAHA", pd.Series(index=df.index, dtype=float)), errors="coerce"),
        })
        odds_frames.append(ah)

    cdf = df.copy()
    if "HC" not in cdf.columns and "HomeCorners" in cdf.columns: cdf["HC"] = cdf["HomeCorners"]
    if "AC" not in cdf.columns and "AwayCorners" in cdf.columns: cdf["AC"] = cdf["AwayCorners"]
    if "HC" not in cdf.columns and "HCA" in cdf.columns: cdf["HC"] = cdf["HCA"]
    if "AC" not in cdf.columns and "ACA" in cdf.columns: cdf["AC"] = cdf["ACA"]
    if {"HC","AC"}.issubset(cdf.columns):
        corners = pd.DataFrame({
            "match_id": m["match_id"],
            "jn_key": m["jn_key"],
            "HC": pd.to_numeric(cdf["HC"], errors="coerce"),
            "AC": pd.to_numeric(cdf["AC"], errors="coerce"),
        })
        corners_frames.append(corners)

len(matches_frames), len(odds_frames), len(corners_frames)


/var/folders/xy/fs1cvf415szgw9x79lyywp1h0000gn/T/ipykernel_70252/567111101.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)
/var/folders/xy/fs1cvf415szgw9x79lyywp1h0000gn/T/ipykernel_70252/567111101.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)
/var/folders/xy/fs1cvf415szgw9x79lyywp1h0000gn/T/ipykernel_70252/567111101.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)
/var/folders/xy/fs1cvf415szg

(57, 128, 57)

In [6]:
matches = pd.concat(matches_frames, ignore_index=True)
matches = matches.sort_values(["Date","Div","HomeTeam","AwayTeam"])
matches = matches.drop_duplicates(subset=["match_id"], keep="first")

odds = pd.concat(odds_frames, ignore_index=True) if odds_frames else pd.DataFrame(columns=["match_id","jn_key"])

agg_map = {
    "pre_home":"first","pre_draw":"first","pre_away":"first",
    "close_home":"first","close_draw":"first","close_away":"first",
    "pre_over2_5":"first","pre_under2_5":"first","close_over2_5":"first","close_under2_5":"first",
    "pre_line_home":"first","close_line_home":"first",
    "pre_home_ah":"first","pre_away_ah":"first","close_home_ah":"first","close_away_ah":"first",
}
if not odds.empty:
    odds = odds.groupby(["match_id","jn_key"], as_index=False).agg(agg_map)

corners = pd.concat(corners_frames, ignore_index=True) if corners_frames else pd.DataFrame(columns=["match_id","jn_key","HC","AC"])
if not corners.empty:
    corners = corners.drop_duplicates(subset=["jn_key"], keep="first")

matches.shape, odds.shape, corners.shape


((18241, 19), (18240, 18), (18241, 4))

In [7]:
merged = matches.merge(odds, on=["match_id","jn_key"], how="left", suffixes=("",""))
merged = merged.merge(corners[["jn_key","HC","AC"]], on="jn_key", how="left")
merged["total_corners"] = merged["HC"] + merged["AC"]

probs = implied_probs_1x2(merged)
merged = pd.concat([merged, probs], axis=1)
merged["p_over25"] = implied_prob_over25(merged)

cols_first = ["match_id","Div","Season","Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HC","AC","total_corners"]
odds_cols = ["pre_home","pre_draw","pre_away","close_home","close_draw","close_away",
             "pre_over2_5","pre_under2_5","close_over2_5","close_under2_5",
             "pre_line_home","close_line_home","pre_home_ah","pre_away_ah","close_home_ah","close_away_ah"]
prob_cols = ["p_home","p_draw","p_away","p_over25"]
meta_cols = ["HTHG","HTAG","HTR","Attendance","Referee","__source_file","jn_date","jn_home","jn_away","jn_key"]
final_cols = [c for c in cols_first + odds_cols + prob_cols + meta_cols if c in merged.columns]
merged = merged[final_cols]

print("Merged shape:", merged.shape)
merged.head(10)


Merged shape: (18241, 42)


,match_id,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HC,AC,total_corners,pre_home,pre_draw,pre_away,close_home,close_draw,close_away,pre_over2_5,pre_under2_5,close_over2_5,close_under2_5,pre_line_home,close_line_home,pre_home_ah,pre_away_ah,close_home_ah,close_away_ah,p_home,p_draw,p_away,p_over25,HTHG,HTAG,HTR,Attendance,Referee,__source_file,jn_date,jn_home,jn_away,jn_key
0,3ee520466726d91f,F1,NaN,2015-08-07,Lille,Paris SG,0.0,1.0,A,3.0,2.0,5.0,6.50,3.6,1.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,D,NaN,NaN,F1 (10).csv,2015-08-07,lille,paris sg,2015-08-07|lille|paris sg
1,92ff78b3164950d0,E0,NaN,2015-08-08,Bournemouth,Aston Villa,0.0,1.0,A,6.0,3.0,9.0,2.00,3.6,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,D,NaN,M Clattenburg,E0 (10).csv,2015-08-08,bournemouth,aston villa,2015-08-08|bournemouth|aston villa
2,481f9a32684b2ed6,E0,NaN,2015-08-08,Chelsea,Swansea,2.0,2.0,D,4.0,8.0,12.0,1.36,5.0,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,H,NaN,M Oliver,E0 (10).csv,2015-08-08,chelsea,swansea,2015-08-08|chelsea|swansea
3,5cd2e61c3f58beae,E0,NaN,2015-08-08,Everton,Watford,2.0,2.0,D,8.0,2.0,10.0,1.70,3.9,5.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,A,NaN,M Jones,E0 (10).csv,2015-08-08,everton,watford,2015-08-08|everton|watford
4,c0470cbc33f6e441,E0,NaN,2015-08-08,Leicester,Sunderland,4.0,2.0,H,6.0,3.0,9.0,1.95,3.5,4.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,H,NaN,L Mason,E0 (10).csv,2015-08-08,leicester,sunderland,2015-08-08|leicester|sunderland
5,4af88950f62e3288,E0,NaN,2015-08-08,Man United,Tottenham,1.0,0.0,H,1.0,2.0,3.0,1.65,4.0,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,H,NaN,J Moss,E0 (10).csv,2015-08-08,man united,tottenham,2015-08-08|man united|tottenham
6,bf8179247a048d35,E0,NaN,2015-08-08,Norwich,Crystal Palace,1.0,3.0,A,1.0,4.0,5.0,2.55,3.3,3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,A,NaN,S Hooper,E0 (10).csv,2015-08-08,norwich,crystal palace,2015-08-08|norwich|crystal palace
7,a4a07af6ebe9f5e8,F1,NaN,2015-08-08,Bastia,Rennes,2.0,1.0,H,6.0,2.0,8.0,2.60,3.1,2.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,A,NaN,NaN,F1 (10).csv,2015-08-08,bastia,rennes,2015-08-08|bastia|rennes
8,e055079d3c247b12,F1,NaN,2015-08-08,Marseille,Caen,0.0,1.0,A,14.0,2.0,16.0,1.57,4.0,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,A,NaN,NaN,F1 (10).csv,2015-08-08,marseille,caen,2015-08-08|marseille|caen
9,3a76c751cd357549,F1,NaN,2015-08-08,Montpellier,Angers,0.0,2.0,A,5.0,9.0,14.0,1.83,3.4,4.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,A,NaN,NaN,F1 (10).csv,2015-08-08,montpellier,angers,2015-08-08|montpellier|angers


In [8]:
merged.to_csv(OUTPUT_MERGED, index=False)
print("Saved:", OUTPUT_MERGED)


Saved: data/b365_merged.csv
